# Reflection and Blogpost Writing

In [1]:
from utils import get_openai_api_key

import autogen

/home/muriloms/Documents/rep/ai-agent-projects/autogen/venv/lib/python3.11/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


In [2]:
OPEN_AI_KEY = get_openai_api_key()
llm_config={
    "model":"gpt-4o-mini"
}

### The task

In [3]:
task = '''
    Write a concise but engaging blogpost about Dragon Ball Z. 
    Make sure the blogpost is within 100 words
'''

### Create a writer agent

In [4]:
writer = autogen.AssistantAgent(
    name='Writer',
    system_message="You are a writer. You write engaging and concise blogpost (with title) on given topics."
    "You must polish your writing based on the feedback you receive and give a refined version"
    "Only return your final work without additional comments.",
    llm_config=llm_config
)

[autogen.oai.client: 01-14 13:51:29] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [5]:
reply = writer.generate_reply(
    messages=[
        {
            'content':task,
            'role':"user"
        }
    ]
)

In [6]:
print(reply)

**The Legacy of Dragon Ball Z: A Timeless Classic**

Dragon Ball Z, a cornerstone of anime, continues to captivate audiences with its exhilarating battles and unforgettable characters. Following Goku and his allies, this iconic series expertly blends action, friendship, and the pursuit of strength. Each epic transformation, from Super Saiyan to Ultra Instinct, pushes the boundaries of imagination, while villains like Frieza and Majin Buu keep viewers on the edge of their seats. With its rich lore and inspirational themes, Dragon Ball Z has not only shaped a generation of fans but also remains a powerful influence in the realms of storytelling and animation today.


### Adding reflection
Create a critic agent to reflect on the work of the writer agent

In [7]:
critic = autogen.AssistantAgent(
    name='Critic',
    is_termination_msg=lambda x: x.get('content',"").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of the writer and provide constructive "
    "feedback to help improve the quality of the content.",
)

[autogen.oai.client: 01-14 13:56:05] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [8]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


    Write a concise but engaging blogpost about Dragon Ball Z. 
    Make sure the blogpost is within 100 words


--------------------------------------------------------------------------------
Writer (to Critic):

**The Legacy of Dragon Ball Z: A Timeless Classic**

Dragon Ball Z, a cornerstone of anime, continues to captivate audiences with its exhilarating battles and unforgettable characters. Following Goku and his allies, this iconic series expertly blends action, friendship, and the pursuit of strength. Each epic transformation, from Super Saiyan to Ultra Instinct, pushes the boundaries of imagination, while villains like Frieza and Majin Buu keep viewers on the edge of their seats. With its rich lore and inspirational themes, Dragon Ball Z has not only shaped a generation of fans but also remains a powerful influence in the realms of storytelling and animation today.

--------------------------------------------------------------------------------
Critic (t

### Nested chat

In [9]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

[autogen.oai.client: 01-14 14:02:10] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [10]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

[autogen.oai.client: 01-14 14:02:28] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [11]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

[autogen.oai.client: 01-14 14:02:33] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [12]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

[autogen.oai.client: 01-14 14:03:06] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


### Orchestrate the nested chats to solve the task

In [13]:
def reflection_message(
        recipient,
        messages,
        sender,
        config
):
    reflec_msg = f'''Review the following content.
                \n\n{recipient.chat_messages_for_summary(sender)[-1]['content']}
                '''

In [14]:
review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1
     },
    {
    "recipient": legal_reviewer, 
    "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1
     },
    {
        "recipient": ethics_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1
     },
     {
      "recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1
     },
]

In [15]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [16]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


    Write a concise but engaging blogpost about Dragon Ball Z. 
    Make sure the blogpost is within 100 words


--------------------------------------------------------------------------------
Writer (to Critic):

**The Legacy of Dragon Ball Z: A Timeless Classic**

Dragon Ball Z, a cornerstone of anime, continues to captivate audiences with its exhilarating battles and unforgettable characters. Following Goku and his allies, this iconic series expertly blends action, friendship, and the pursuit of strength. Each epic transformation, from Super Saiyan to Ultra Instinct, pushes the boundaries of imagination, while villains like Frieza and Majin Buu keep viewers on the edge of their seats. With its rich lore and inspirational themes, Dragon Ball Z has not only shaped a generation of fans but also remains a powerful influence in the realms of storytelling and animation today.

--------------------------------------------------------------------------------

********

In [17]:
print(res.summary)

**The Legacy of Dragon Ball Z: A Timeless Classic**

Dragon Ball Z is a cornerstone of anime that continues to captivate audiences worldwide. Following Goku and his allies, this iconic series expertly blends action, friendship, and the relentless pursuit of strength. Each epic transformation, from Super Saiyan to Ultra Instinct, pushes creative boundaries, while iconic villains like Frieza and Majin Buu keep fans on the edge of their seats. Its rich lore and inspirational themes have shaped generations, making Dragon Ball Z not just a show, but a cultural phenomenon that influences storytelling and animation today. Dive into the world of Z and experience the adventure!
